In [4]:
!pip install lupyne
!pip install Cyton

ERROR: Could not find a version that satisfies the requirement Cyton (from versions: none)
ERROR: No matching distribution found for Cyton


In [ ]:
import csv

# Assuming the CSV file is named 'wapo.csv'
with open('wapo.csv', 'r', newline='', encoding='utf-16') as csvfile:
  reader = csv.reader(csvfile, delimiter=',', quotechar='"')  # Double quotes as delimiters

  # Skip the header row (optional)
  next(reader)

  # List to store extracted data
  data = []
  for row in reader:
    if len(row) >= 2 and "\t\t\t" not in row[0]: 
        # Extract id and title from each row
        id = row[0]
        title = row[1].strip()  # Remove any leading/trailing whitespaces from title

        # Append data to the list
        data.append({'id': id, 'title': title})

# Now you have a list of dictionaries containing 'id' and 'title'
print(len(data))


# Inverted Index Implementation

In [ ]:
import csv
import re as re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# removes the suffixes from an English word and obtain its stem
porter_stemmer = nltk.stem.PorterStemmer()

def tokenize(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    #tokens = text.split()  # Split into tokens
    # tokens = [token for token in text.split() if token not in stop_words]
    
    # keep 1st, 2nd, 3rd, 24st (ordinal numbers) etc but separate words like 10monthold, 10year, 10yearold, 20point, 20game, 23march, 24hour etc.
    text = re.sub(r'(?<!\d)(\d+)(?!st|nd|rd|th)(?!\d)', r'\1 ', text)  # Add space between digits and letters except for ordinal numbers

    tokens = [porter_stemmer.stem(token) for token in text.split() if token not in stop_words]

    return tokens

def build_inverted_index(file_path):
    inverted_index = {}
    doc_lengths = {}  # Dictionary to store document lengths
    doc_titles = {}  # Dictionary to store document titles
    
    with open(file_path, 'r', encoding='utf-16') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) >= 2 and "\t\t\t" not in row[0]: 
                doc_id = row[0]  # Assuming doc_id is in the first column
                title = row[1]   # Assuming title is in the second column
                
                # Tokenize title
                tokens = tokenize(title)
                
                # Update document lengths
                doc_lengths[doc_id] = len(tokens)
                
                # Update inverted index
                for token in tokens:
                    if token not in inverted_index:
                        inverted_index[token] = {'doc_freq': 0, 'postings': {}}
                    
                    if doc_id not in inverted_index[token]['postings']:
                        inverted_index[token]['postings'][doc_id] = 0
                    
                    inverted_index[token]['postings'][doc_id] += 1
                    inverted_index[token]['doc_freq'] += 1

                # Update document titles
                doc_titles[doc_id] = title

    # Sort terms alphabetically
    sorted_index = {}
    for term in sorted(inverted_index):
        sorted_index[term] = inverted_index[term]
    
    return sorted_index, doc_lengths, doc_titles

# def main():
#     file_path = 'wapo.csv'
#     inverted_index, _ = build_inverted_index(file_path)
# 
#     # Print the inverted index
#     sorted_terms = sorted(inverted_index.keys())
#     for term in sorted_terms[2000:2200]:  # Print only the first 10 terms
#         print(f'{term}: {inverted_index[term]}')

file_path = 'wapo.csv'
inverted_index, doc_lengths, doc_titles = build_inverted_index(file_path)

import json

# Save the inverted index, document lengths, and document titles to files
def save_to_files(inverted_index, doc_lengths, doc_titles):
    with open('inverted_index.json', 'w') as f:
        json.dump(inverted_index, f)
    
    with open('doc_lengths.json', 'w') as f:
        json.dump(doc_lengths, f)
    
    with open('doc_titles.json', 'w') as f:
        json.dump(doc_titles, f)

# Save the data to files
save_to_files(inverted_index, doc_lengths, doc_titles)

# Write the inverted index to a text file
with open('inverted_index.txt', 'w') as f:
    sorted_terms = sorted(inverted_index.keys())
    for term in sorted_terms[0:5000]: 
        f.write(f'{term}: {inverted_index[term]}\n')



## Load Inverted Index

In [ ]:
# Load the inverted index, document lengths, and document titles from files
def load_from_files():
    with open('inverted_index.json', 'r') as f:
        inverted_index = json.load(f)
    
    with open('doc_lengths.json', 'r') as f:
        doc_lengths = json.load(f)
    
    with open('doc_titles.json', 'r') as f:
        doc_titles = json.load(f)
    
    return inverted_index, doc_lengths, doc_titles

# Load the data from files
inverted_index, doc_lengths, doc_titles = load_from_files()

# Verify loaded data
print(f"Loaded inverted index sample: {list(inverted_index.items())[:5]}")
print(f"Loaded document lengths sample: {list(doc_lengths.items())[:5]}")
print(f"Loaded document titles sample: {list(doc_titles.items())[:5]}")

In [ ]:
def calculate_stats(inverted_index, doc_lengths):
    num_documents = len(doc_lengths)
    
    num_tokens = len(inverted_index)
    
    total_words = sum(doc_lengths.values())
    
    return num_documents, num_tokens, total_words

def calculate_average_words_per_document(doc_lengths):
    total_words = sum(doc_lengths.values())
    total_documents = len(doc_lengths)
    average_words = total_words / total_documents if total_documents > 0 else 0
    return average_words

average_words = calculate_average_words_per_document(doc_lengths)

print(f'Average number of words per document: {average_words}')

num_documents, num_tokens, total_words = calculate_stats(inverted_index, doc_lengths)
print(f"Number of documents: {num_documents}")
print(f"Number of tokens: {num_tokens}")
print(f"Total words: {total_words}")

## Preprocess Queries

In [ ]:
def preprocess_queries(query_file):
    with open(query_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content into individual queries
    queries = content.split('<top>')
    preprocessed_queries = []

    for query in queries[1:]:  # Skip the first empty string
        query_parts = query.strip().split('\n')

        # Find the <desc> and </desc> tags and extract the text in between
        desc_start = None
        desc_end = None
        for i, part in enumerate(query_parts):
            if part.startswith('<desc> Description:'):
                desc_start = i + 1  # Start from the next line after <desc> tag
            elif part.startswith('</desc>'):
                desc_end = i  # End at the line before </desc> tag
                break

        if desc_start is not None and desc_end is not None:
            query_num = re.search(r'Number: (\d+)', query_parts[0]).group(1)
            query_text = ' '.join([part.strip() for part in query_parts[desc_start:desc_end]])
            tokens = tokenize(query_text)
            preprocessed_queries.append((query_num, ' '.join(tokens)))

    return preprocessed_queries

# Preprocess the queries
query_file = 'topics2018.txt'
preprocessed_queries = preprocess_queries(query_file)

# Print the preprocessed queries
for query_num, preprocessed_query in preprocessed_queries:
    print(f"Query {query_num}: {preprocessed_query}")


# BM25

In [ ]:
import math

class BM25:
    def __init__(self, inverted_index, doc_lengths, k1=1.5, b=0.75):
        self.inverted_index = inverted_index
        self.doc_lengths = doc_lengths
        self.avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)
        self.k1 = k1
        self.b = b
        self.N = len(doc_lengths)
        self.idf_cache = {}

    def idf(self, term):
        if term not in self.idf_cache:
            df = self.inverted_index.get(term, {'doc_freq': 0})['doc_freq']
            self.idf_cache[term] = math.log((self.N - df + 0.5) / (df + 0.5) + 1)
        return self.idf_cache[term]

    def score(self, query_terms, doc_id):
        score = 0
        doc_length = self.doc_lengths[doc_id]
        for term in query_terms:
            tf = self.inverted_index.get(term, {'postings': {}})['postings'].get(doc_id, 0)
            idf = self.idf(term)
            score += idf * (tf * (self.k1 + 1)) / (tf + self.k1 * (1 - self.b + self.b * doc_length / self.avg_doc_length))
        return score

def evaluate_queries(queries, relevant_docs, inverted_index, doc_lengths, k=10):
    bm25 = BM25(inverted_index, doc_lengths)
    average_precision = 0
    precision_at_10 = 0
    ndcg_at_10 = 0
    total_queries = len(queries)

    for query_num, query_text in queries:
        query_terms = query_text.split()
        scores = [(doc_id, bm25.score(query_terms, doc_id)) for doc_id in doc_lengths.keys()]
        ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)[:k]

        relevant_docs_for_query = relevant_docs.get(query_num, [])
        precision_count = sum(1 for doc_id, _ in ranked_docs[:k] if doc_id in relevant_docs_for_query)
        precision_at_10 += precision_count / k

        average_precision += compute_average_precision(ranked_docs, relevant_docs_for_query)
        ndcg_at_10 += compute_ndcg_at_k(ranked_docs, relevant_docs_for_query, k)

    map_score = average_precision / total_queries
    precision_at_10_score = precision_at_10 / total_queries
    ndcg_at_10_score = ndcg_at_10 / total_queries

    return map_score, precision_at_10_score, ndcg_at_10_score

def compute_average_precision(ranked_docs, relevant_docs):
    num_relevant_docs = len(relevant_docs)
    if num_relevant_docs == 0:
        return 0
    
    precision_sum = 0
    num_retrieved_relevant_docs = 0
    for i, (doc_id, _) in enumerate(ranked_docs, start=1):
        if doc_id in relevant_docs:
            num_retrieved_relevant_docs += 1
            precision_sum += num_retrieved_relevant_docs / i
    
    return precision_sum / num_relevant_docs

def compute_ndcg_at_k(ranked_docs, relevant_docs, k):
    if not relevant_docs:
        return 0
    
    dcg = 0
    idcg = 0
    # for i, (doc_id, _) in enumerate(ranked_docs[:k], start=1):
        # relevance = relevant_docs.get(doc_id, 0)
        # gain = (relevance) / math.log2(i + 1)
        # dcg += gain
        # idcg_gain = (max(relevant_docs.values())) / math.log2(i + 1)
        # idcg += idcg_gain
    
    for i, (doc_id, _) in enumerate(ranked_docs[:k], start=1):
        if doc_id in relevant_docs:
            dcg += 1 / math.log2(i + 1)
        idcg += 1 / math.log2(i + 1)

    return dcg / idcg

def load_relevant_docs(relevant_docs_file):
    relevant_docs = {}
    with open(relevant_docs_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            query_num = parts[0]
            doc_id = parts[2]
            relevance = int(parts[3])  # 0 for non-relevant, 1 for relevant, 2 for highly relevant
            if relevance >= 0:  # considering all relevance levels
                if query_num in relevant_docs:
                    relevant_docs[query_num][doc_id] = relevance
                else:
                    relevant_docs[query_num] = {doc_id: relevance}
    return relevant_docs


query_file = 'topics2018.txt'
queries = preprocess_queries(query_file)

relevant_docs_file = 'qrels2018.txt'  # File containing ground truth relevance judgments
relevant_docs = load_relevant_docs(relevant_docs_file)

map_score, precision_at_10_score, ndcg_at_10_score = evaluate_queries(queries, relevant_docs, inverted_index, doc_lengths)

print(f'Mean Average Precision (MAP): {map_score}')
print(f'Precision@10: {precision_at_10_score}')
print(f'NDCG@10: {ndcg_at_10_score}')


## Test BM25

In [ ]:
def apply_bm25_to_query(query, inverted_index, doc_lengths, k=20):
    bm25 = BM25(inverted_index, doc_lengths)
    query_terms = tokenize(query)
    scores = [(doc_id, bm25.score(query_terms, doc_id)) for doc_id in doc_lengths.keys()]
    ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)[:k]
    return ranked_docs

# Example usage:
query = "How does climate change affect weather patterns?"
query = "Map shows changes to climate"
ranked_docs = apply_bm25_to_query(query, inverted_index, doc_lengths)
print("Ranked Documents:")
for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
    print(f"{rank}. Document ID: {doc_id}, Score: {score}, {doc_titles[doc_id]}")

# Query Likelihood

In [ ]:
import math
import re
""
class QueryLikelihoodModel:
    def __init__(self, inverted_index, doc_lengths, smoothing_param=0.5):
        self.inverted_index = inverted_index
        self.doc_lengths = doc_lengths
        self.smoothing_param = smoothing_param
        self.total_terms = sum(doc_lengths.values())
        self.term_freq_cache = {}

    def term_freq(self, term, doc_id):
        if (term, doc_id) not in self.term_freq_cache:
            tf = self.inverted_index.get(term, {'postings': {}})['postings'].get(doc_id, 0)
            doc_length = self.doc_lengths[doc_id]
            self.term_freq_cache[(term, doc_id)] = (tf + self.smoothing_param) / (doc_length + self.smoothing_param * len(self.inverted_index))
        return self.term_freq_cache[(term, doc_id)]

    def coll_freq(self, term):
        return self.inverted_index.get(term, {'doc_freq': 0})['doc_freq']

    def score(self, query_terms, doc_id):
        score = 0
        for term in query_terms:
            term_freq = self.term_freq(term, doc_id)
            coll_freq = self.coll_freq(term) / self.total_terms
            
            # Add a small positive value (e.g., 1e-10) to prevent log(0)
            score += math.log(term_freq + 1e-10) - math.log(coll_freq + 1e-10)
        return score

def compute_average_precision(ranked_docs, relevant_docs):
    num_relevant_docs = len(relevant_docs)
    if num_relevant_docs == 0:
        return 0
    
    precision_sum = 0
    num_retrieved_relevant_docs = 0
    for i, (doc_id, _) in enumerate(ranked_docs, start=1):
        if doc_id in relevant_docs:
            num_retrieved_relevant_docs += 1
            precision_sum += num_retrieved_relevant_docs / i
    
    return precision_sum / num_relevant_docs

def compute_ndcg_at_k(ranked_docs, relevant_docs, k):
    if not relevant_docs:
        return 0
    
    dcg = 0
    idcg = 0
    # for i, (doc_id, _) in enumerate(ranked_docs[:k], start=1):
        # relevance = relevant_docs.get(doc_id, 0)
        # gain = (relevance) / math.log2(i + 1)
        # dcg += gain
        # idcg_gain = (max(relevant_docs.values())) / math.log2(i + 1)
        # idcg += idcg_gain
    
    for i, (doc_id, _) in enumerate(ranked_docs[:k], start=1):
        if doc_id in relevant_docs:
            dcg += 1 / math.log2(i + 1)
        idcg += 1 / math.log2(i + 1)

    return dcg / idcg

def load_relevant_docs(relevant_docs_file):
    relevant_docs = {}
    with open(relevant_docs_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            query_num = parts[0]
            doc_id = parts[2]
            relevance = int(parts[3])  # 0 for non-relevant, 1 for relevant, 2 for highly relevant
            if relevance >= 0:  # considering all relevance levels
                if query_num in relevant_docs:
                    relevant_docs[query_num][doc_id] = relevance
                else:
                    relevant_docs[query_num] = {doc_id: relevance}
    return relevant_docs

def evaluate_queries_ql(queries, relevant_docs, inverted_index, doc_lengths, k=10):
    qln_model = QueryLikelihoodModel(inverted_index, doc_lengths)
    average_precision = 0
    precision_at_10 = 0
    ndcg_at_10 = 0
    total_queries = len(queries)

    for query_num, query_text in queries:
        query_terms = query_text.split()
        scores = [(doc_id, qln_model.score(query_terms, doc_id)) for doc_id in doc_lengths.keys()]
        ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)[:k]

        relevant_docs_for_query = relevant_docs.get(query_num, [])
        precision_count = sum(1 for doc_id, _ in ranked_docs[:k] if doc_id in relevant_docs_for_query)
        precision_at_10 += precision_count / k

        average_precision += compute_average_precision(ranked_docs, relevant_docs_for_query)
        ndcg_at_10 += compute_ndcg_at_k(ranked_docs, relevant_docs_for_query, k)

    map_score = average_precision / total_queries
    precision_at_10_score = precision_at_10 / total_queries
    ndcg_at_10_score = ndcg_at_10 / total_queries

    return map_score, precision_at_10_score, ndcg_at_10_score


query_file = 'topics2018.txt'
queries = preprocess_queries(query_file)

relevant_docs_file = 'qrels2018.txt'  # File containing ground truth relevance judgments
relevant_docs = load_relevant_docs(relevant_docs_file)

map_score, precision_at_10_score, ndcg_at_10_score = evaluate_queries_ql(queries, relevant_docs, inverted_index, doc_lengths)
    
print(f'Mean Average Precision (MAP): {map_score}')
print(f'Precision@10: {precision_at_10_score}')
print(f'NDCG@10: {ndcg_at_10_score}')



# Phrase Query

In [ ]:
import csv
import re

def build_inverted_index_with_position(file_path):
    inverted_index = {}
    doc_lengths = {}  # Dictionary to store document lengths
    doc_titles = {}  # Dictionary to store document titles
    
    with open(file_path, 'r', encoding='utf-16') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) >= 2 and "\t\t\t" not in row[0]: 
                doc_id = row[0]  # Assuming doc_id is in the first column
                title = row[1]   # Assuming title is in the second column
                
                # Tokenize title
                tokens = tokenize(title)
                
                # Update document lengths
                doc_lengths[doc_id] = len(tokens)
                
                # Update inverted index with positional information
                for pos, token in enumerate(tokens):
                    if token not in inverted_index:
                        inverted_index[token] = {'doc_freq': 0, 'postings': {}}
                    
                    if doc_id not in inverted_index[token]['postings']:
                        inverted_index[token]['postings'][doc_id] = []
                    
                    inverted_index[token]['postings'][doc_id].append(pos)
                    inverted_index[token]['doc_freq'] += 1

                # Update document titles
                doc_titles[doc_id] = title

    # Sort terms alphabetically
    sorted_index = {}
    for term in sorted(inverted_index):
        sorted_index[term] = inverted_index[term]
    
    return sorted_index, doc_lengths, doc_titles

def phrase_query(inverted_index, query):
    query_terms = tokenize(query)
    doc_scores = {}
    
    # Initialize document scores
    for term in query_terms:
        if term in inverted_index:
            for doc_id, positions in inverted_index[term]['postings'].items():
                if doc_id not in doc_scores:
                    doc_scores[doc_id] = 0
    
    # Calculate scores based on phrase proximity
    for term in query_terms:
        if term in inverted_index:
            for doc_id, positions in inverted_index[term]['postings'].items():
                if all(doc_id in inverted_index[qt]['postings'] for qt in query_terms):
                    for pos in positions:
                        match_count = 0
                        for qt in query_terms:
                            if pos + query_terms.index(qt) in inverted_index[qt]['postings'][doc_id]:
                                match_count += 1
                        if match_count == len(query_terms):
                            doc_scores[doc_id] += 1
    
    # Sort documents by score
    sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_docs

# Example usage:
file_path = 'wapo.csv'
inverted_index_pq, _, doc_titles_pq = build_inverted_index_pq(file_path)
query = "As homicides"
query = "Danny Coale"

results = phrase_query(inverted_index_pq, query)

# Print top 10 results
for doc_id, score in results[:10]:
    if score == 1:
        print(f"Document ID: {doc_id}, Score: {score}, {doc_titles_pq[doc_id]}")

# Boolean Query

In [ ]:
def parse_boolean_query(input):
    """
    Parse a boolean query string and extract terms and operation.

    :param input: Boolean query string
    :return: Two terms extracted from the query string and the operation
    """
    # Split the input string based on the operation keywords
    if "AND" in input:
        operation = "AND"
        terms = input.split("AND")
    elif "OR" in input:
        operation = "OR"
        terms = input.split("OR")
    elif "NOT" in input:
        operation = "NOT"
        terms = input.split("NOT")
    
    # Strip leading and trailing spaces from each term and the operation
    term1 = terms[0].strip()
    term2 = terms[1].strip()
    
    return term1, term2, operation

def get_term_postings_set(inverted_index, term):
    if term in inverted_index:
        return set(inverted_index[term]['postings'].keys())
    else:
        # If the term is not in the index, return an empty set
        return set()

def process_and_boolean_query(inverted_index, term1, term2):
    # Retrieve sets for each term
    term1_postings = get_term_postings_set(inverted_index, term1)
    term2_postings = get_term_postings_set(inverted_index, term2)
    
    # Find the intersection of the two sets
    intersection = term1_postings.intersection(term2_postings)
    
    return term1_postings, term2_postings, intersection

def process_or_boolean_query(inverted_index, term1, term2):
    # Retrieve sets for each term
    term1_postings = get_term_postings_set(inverted_index, term1)
    term2_postings = get_term_postings_set(inverted_index, term2)
    
    # Find the union of the two sets
    union = term1_postings.union(term2_postings)
    
    return term1_postings, term2_postings, union

def process_not_boolean_query(inverted_index, term):
    # Retrieve the set of document IDs for the given term
    term_postings = get_term_postings_set(inverted_index, term)
    
    # Retrieve all document IDs in the index
    all_doc_ids = set()
    for postings in inverted_index.values():
        all_doc_ids.update(postings['postings'].keys())
    
    # Find the set of document IDs that do not contain the term
    not_term_postings = all_doc_ids - term_postings
    
    return not_term_postings

def process_boolean_query(query):
    parsed_term1, parsed_term2, operation = parse_boolean_query(query)
    if(operation == "AND"):
        term1 = tokenize(parsed_term1)[0]
        term2 = tokenize(parsed_term2)[0]
        term1_postings, term2_postings, intersection = process_and_boolean_query(inverted_index, term1, term2)
        return intersection

    elif operation == "OR":
        term1 = tokenize(parsed_term1)[0]
        term2 = tokenize(parsed_term2)[0]
        term1_postings, term2_postings, union = process_or_boolean_query(inverted_index, term1, term2)
        return union

    elif operation == "NOT":
        term = tokenize(parsed_term2)[0]
        not_term_postings = process_not_boolean_query(inverted_index, term)
        return not_term_postings


query = "Maryland AND win AND blair"
results = process_boolean_query(query)

counter = 0
for doc_id in results:
    if counter >= 10:
        break
    title = doc_titles.get(doc_id, "Title not available")
    print(f"Document ID: {doc_id} {title}")  
    counter += 1

# Extended Boolean Query

In [ ]:
import json
import nltk
import collections

# Load inverted index and document titles from files
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

with open('doc_titles.json', 'r') as f:
    doc_titles = json.load(f)

def process_query(query, inverted_index, doc_titles):
    stemmer = nltk.stem.porter.PorterStemmer()  # Use the PorterStemmer for consistency with your indexing
    indexed_docIDs = sorted(doc_titles.keys())  # All docIDs from your titles
    
    # Prepare query list
    query = query.replace('(', '( ')
    query = query.replace(')', ' )')
    query = query.split(' ')
    
    results_stack = []
    postfix_queue = collections.deque(shunting_yard(query))  # Convert infix to postfix
    
    while postfix_queue:
        token = postfix_queue.popleft()
        result = []
        
        if token not in ['AND', 'OR', 'NOT']:
            token = stemmer.stem(token)
            if token in inverted_index:
                result = list(inverted_index[token]['postings'].keys())
        elif token == 'AND':
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = boolean_AND(left_operand, right_operand, indexed_docIDs)
        elif token == 'OR':
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = boolean_OR(left_operand, right_operand, indexed_docIDs)
        elif token == 'NOT':
            right_operand = results_stack.pop()
            result = boolean_NOT(right_operand, indexed_docIDs)
        
        results_stack.append(result)
    
    return results_stack.pop()

def shunting_yard(infix_tokens):
    precedence = {'NOT': 3, 'AND': 2, 'OR': 1, '(': 0, ')': 0}
    output = []
    operator_stack = []
    
    for token in infix_tokens:
        if token == '(':
            operator_stack.append(token)
        elif token == ')':
            while operator_stack and operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.pop()  # Pop '('
        elif token in precedence:
            while operator_stack and precedence[operator_stack[-1]] >= precedence[token]:
                output.append(operator_stack.pop())
            operator_stack.append(token)
        else:
            output.append(token)
    
    while operator_stack:
        output.append(operator_stack.pop())
    
    return output

def boolean_AND(left_operand, right_operand, indexed_docIDs):
    left_set = set(left_operand)
    right_set = set(right_operand)
    return list(left_set.intersection(right_set))

def boolean_OR(left_operand, right_operand, indexed_docIDs):
    left_set = set(left_operand)
    right_set = set(right_operand)
    return list(left_set.union(right_set))

def boolean_NOT(right_operand, indexed_docIDs):
    return list(set(indexed_docIDs) - set(right_operand))

# Example query
query = "Maryland AND (win OR NOT financial) AND Bowie"
query = "Maryland AND (win OR NOT financial) AND Bowie AND NOT overmatched"
query = "Maryland AND (Baseball OR Ivey) AND Bowie AND NOT overmatched"

query = "NOT Eleanor AND (win OR NOT financial) AND (Maryland AND Bowie AND NOT Sherwood)"
query = "Ivey AND Brown"
query = "Ivey AND Brown AND NOT 2014"
results = process_query(query, inverted_index, doc_titles)

# Print first 10 results
counter = 0
for doc_id in results:
    if counter >= 10:
        break
    title = doc_titles.get(doc_id, "Title not available")
    print(f"Document ID: {doc_id} Title: {title}")
    counter += 1


In [5]:
import json
import nltk
import collections
import tkinter as tk
from tkinter import ttk

# Load inverted index and document titles from files
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

with open('doc_titles.json', 'r') as f:
    doc_titles = json.load(f)

def process_query(query, inverted_index, doc_titles):
    stemmer = nltk.stem.porter.PorterStemmer()  # Use the PorterStemmer for consistency with your indexing
    indexed_docIDs = sorted(doc_titles.keys())  # All docIDs from your titles
    
    # Prepare query list
    query = query.replace('(', '( ')
    query = query.replace(')', ' )')
    query = query.split(' ')
    
    results_stack = []
    postfix_queue = collections.deque(shunting_yard(query))  # Convert infix to postfix
    
    while postfix_queue:
        token = postfix_queue.popleft()
        result = []
        
        if token not in ['AND', 'OR', 'NOT']:
            token = stemmer.stem(token)
            if token in inverted_index:
                result = list(inverted_index[token]['postings'].keys())
        elif token == 'AND':
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = boolean_AND(left_operand, right_operand, indexed_docIDs)
        elif token == 'OR':
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = boolean_OR(left_operand, right_operand, indexed_docIDs)
        elif token == 'NOT':
            right_operand = results_stack.pop()
            result = boolean_NOT(right_operand, indexed_docIDs)
        
        results_stack.append(result)
    
    return results_stack.pop()

def shunting_yard(infix_tokens):
    precedence = {'NOT': 3, 'AND': 2, 'OR': 1, '(': 0, ')': 0}
    output = []
    operator_stack = []
    
    for token in infix_tokens:
        if token == '(':
            operator_stack.append(token)
        elif token == ')':
            while operator_stack and operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.pop()  # Pop '('
        elif token in precedence:
            while operator_stack and precedence[operator_stack[-1]] >= precedence[token]:
                output.append(operator_stack.pop())
            operator_stack.append(token)
        else:
            output.append(token)
    
    while operator_stack:
        output.append(operator_stack.pop())
    
    return output

def boolean_AND(left_operand, right_operand, indexed_docIDs):
    left_set = set(left_operand)
    right_set = set(right_operand)
    return list(left_set.intersection(right_set))

def boolean_OR(left_operand, right_operand, indexed_docIDs):
    left_set = set(left_operand)
    right_set = set(right_operand)
    return list(left_set.union(right_set))

def boolean_NOT(right_operand, indexed_docIDs):
    return list(set(indexed_docIDs) - set(right_operand))

def search_query():
    query = query_entry.get()
    results = process_query(query, inverted_index, doc_titles)
    results_list.delete(*results_list.get_children())
    counter = 0
    for doc_id in results:
        if counter >= 10:
            break
        title = doc_titles.get(doc_id, "Title not available")
        results_list.insert("", "end", values=(doc_id, title))
        counter += 1

# Create the main window
root = tk.Tk()
root.title("Boolean Query Search")
root.geometry("600x400")

# Create and place the query input field
query_label = tk.Label(root, text="Enter Query:")
query_label.pack(pady=10)
query_entry = tk.Entry(root, width=80)
query_entry.pack(pady=5)

# Create and place the search button
search_button = tk.Button(root, text="Search", command=search_query)
search_button.pack(pady=5)

# Create and place the results treeview
columns = ("Document ID", "Document Title")
results_list = ttk.Treeview(root, columns=columns, show="headings")
results_list.heading("Document ID", text="Document ID")
results_list.heading("Document Title", text="Document Title")
results_list.pack(pady=20, fill=tk.BOTH, expand=True)

# Run the application
root.mainloop()


KeyboardInterrupt: 